In [1]:
import os
import glob
import matplotlib.pyplot as plt

import numpy as np
import scipy.misc
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import SGD
from matplotlib.pyplot import imshow
import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import roc_curve, auc

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [2]:
########### insert hyperparameters ################
train_batch_size = 32
val_batch_size = 32
###################################################

base_dir = r'C:\Users\20153761\Documents\TUe\4e jaar\3e kwartiel\BIA'
test_dir = r'C:\Users\20153761\Documents\TUe\4e jaar\3e kwartiel\BIA\test\test'

# dataset parameters
TRAIN_PATH = os.path.join(base_dir, 'train+val', 'train')
VALID_PATH = os.path.join(base_dir, 'train+val', 'valid')
TEST_FILES = glob.glob(test_dir + '\*.tif')
RESCALING_FACTOR = 1./255
IMAGE_SIZE = 96

# instantiate data generators
datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

train_gen = datagen.flow_from_directory(TRAIN_PATH,
                                    target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                    batch_size=train_batch_size,
                                    class_mode='binary')

val_gen = datagen.flow_from_directory(VALID_PATH,
                                    target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                    batch_size=val_batch_size,
                                    class_mode='binary',
                                    shuffle=False)

# form steps
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [3]:
def identity_block_s(X, f, filters, stage, block): 
    #DOES NOT WORK YET
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters = F3, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [4]:
def convolutional_block_s(X, f, filters, stage, block, s = 2):
    # DOES NOT WORK YET
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X = Conv2D(filters = F3, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [5]:
train_gen.labels.astype('uint8')

array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)

In [6]:
train_gen.labels.dtype

dtype('int32')

In [7]:
input_shape=(96, 96, 3)
classes=1

def build_model(hp):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Preprocess input to the correct size
    #X = Resizing(64,64)

    # Zero-Padding
    X = ZeroPadding2D((3,3))(X_input)

    print('Hello World')
    # Stage 1
    X = Conv2D(64, (7,7), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    # Stage 2
    X = convolutional_block_s(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='c')
    
    # Stage 3
    X = convolutional_block_s(X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
    for i in range(hp.Int('n_IDblocks', 1, 5)):
        X = identity_block_s(X, 3, [128,128,512], stage=3, block='b')

    # AVGPOOL
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet12')
    model.compile(SGD(lr=0.01, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory='First_Resnet',
    project_name='Resnet_Check' 
)

tuner.search(x=train_gen,
             y=train_gen.labels.astype('uint8'),
             verbose=2, # just slapping this here bc jupyter notebook. The console out was getting messy.
             epochs=1,
             batch_size=64,
             #callbacks=[tensorboard],  # if you have callbacks like tensorboard, they go here.
             validation_data=(val_gen, val_gen.labels.astype('uint8')))

Hello World
Hello World


Traceback (most recent call last):
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-5619e43bfe45>", line 39, in build_model
    model = Model(inputs = X_input, outputs = X, name='ResNet12')
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 120, in __init__
    self._init_graph_network(inputs, outputs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 203, in _init_g

Hello World


Traceback (most recent call last):
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-5619e43bfe45>", line 39, in build_model
    model = Model(inputs = X_input, outputs = X, name='ResNet12')
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 120, in __init__
    self._init_graph_network(inputs, outputs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 203, in _init_g

Hello World


...

Hello World


Traceback (most recent call last):
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-5619e43bfe45>", line 39, in build_model
    model = Model(inputs = X_input, outputs = X, name='ResNet12')
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 120, in __init__
    self._init_graph_network(inputs, outputs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 203, in _init_g

Hello World


Traceback (most recent call last):
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-5619e43bfe45>", line 39, in build_model
    model = Model(inputs = X_input, outputs = X, name='ResNet12')
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 120, in __init__
    self._init_graph_network(inputs, outputs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\training\tracking\base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\20153761\Anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\engine\functional.py", line 203, in _init_g

RuntimeError: Too many failed attempts to build model.

In [ ]:
# save the model and weights
model_name = 'ResNet12'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

# serialize model to JSON
model_json = model.to_json()
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json) 
    
# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]